In [7]:
import numpy as np
import pickle

In [8]:
def build_trainset(encoding_data, args):
    '''
        divide training set into different tasks
    '''
    scenario_train = []
    for task_id in range(args.num_tasks):
        task = []
        if task_id == 0:
            for sample in encoding_data:
                if sample[1] <= args.num_bases - 1:
                    task.append(sample)
            scenario_train.append(task)
        else:
            for sample in encoding_data:
                if args.num_bases + (task_id - 1) * args.increment - 1 < sample[
                    1] <= args.num_bases + task_id * args.increment - 1:
                    task.append(sample)
            scenario_train.append(task)
    return scenario_train


class Arguments:
    def __init__(self, num_bases, increment, num_tasks):
        self.num_bases = num_bases
        self.increment = increment
        self.num_tasks = num_tasks


args = Arguments(5, 3, 6)
save_path1 = 'encoded__train_data.pkl'  # Encoded data pkl file
with open(save_path1, 'rb') as ff:
    train_data = pickle.load(ff)
    ff.close()
scenario_train = build_trainset(train_data, args)
train_seq = scenario_train[0]


def nem_add(
        task_id,
        features: np.ndarray,
        base_mem_size,
        inc_mem_size
):
    if len(features.shape) != 2:
        raise ValueError(f"Expected features to have 2 dimensions, not {len(features.shape)}d.")

    indexes = []

    if task_id == 0:
        num_classes = 5
        num_samples = 90
        num_per_class = base_mem_size
    else:
        num_classes = 3
        num_samples = 10
        num_per_class = inc_mem_size

    for class_id in range(num_classes):
        class_indexes = np.array(range(class_id * num_samples, (class_id + 1) * num_samples))
        class_features = features[class_indexes]

        D = class_features.T
        D = D / (np.linalg.norm(D, axis=0) + 1e-8)
        mu = np.mean(D, axis=1)
        herding_matrix = np.zeros((class_features.shape[0],))

        w_t = mu
        iter_herding, iter_herding_eff = 0, 0

        while not (
                np.sum(herding_matrix != 0) == min(num_per_class, class_features.shape[0])
        ) and iter_herding_eff < 1000:
            tmp_t = np.dot(w_t, D)
            ind_max = np.argmax(tmp_t)
            iter_herding_eff += 1
            if herding_matrix[ind_max] == 0:
                herding_matrix[ind_max] = 1 + iter_herding
                iter_herding += 1

            w_t = w_t + mu - D[:, ind_max]

        herding_matrix[np.where(herding_matrix == 0)[0]] = 10000

        tmp_indexes = herding_matrix.argsort()[:num_per_class]
        indexes.append(class_indexes[tmp_indexes])

    indexes = np.concatenate(indexes)
    return indexes


def random_add(task_id, base_mem_size, inc_mem_size):
    select_id = []
    if task_id == 0:
        num_classes = 5
        num_samples = 90
        num_per_class = base_mem_size
    else:
        num_classes = 3
        num_samples = 10
        num_per_class = inc_mem_size
    for class_id in range(num_classes):
        sample_id = list(range(class_id * num_samples, (class_id + 1) * num_samples))
        select_id = \
        np.concatenate((select_id, np.random.choice(sample_id, num_per_class)), axis=0)
    return select_id


def n2c_add(task_id, features: np.ndarray, base_mem_size, inc_mem_size):
    indexes = []
    if task_id == 0:
        num_classes = 5
        num_samples = 90
        num_per_class = base_mem_size
    else:
        num_classes = 3
        num_samples = 10
        num_per_class = inc_mem_size

    for class_id in range(num_classes):
        class_indexes = np.array(range(class_id * num_samples, (class_id + 1) * num_samples))
        class_features = features[class_indexes]
        class_mean = np.mean(class_features, axis=0, keepdims=True)

        dist_to_mean = np.linalg.norm(class_mean - class_features, axis=1)
        tmp_indexes = dist_to_mean.argsort()[:num_per_class]

        indexes.append(class_indexes[tmp_indexes])

    indexes = np.concatenate(indexes)
    return indexes

In [10]:
features = np.load('features.npy')
selected_id_nem = nem_add(0, features, 10, 5)
selected_id_n2c = n2c_add(0, features, 10, 5)
selected_id_rand = random_add(0, 10, 5)

In [13]:
def save_as_txt(test_results, train_results, save_path):
    file = open(save_path,'w')
    file.write('Test Results\n')
    for result in test_results:
        file.write(str(result) + '\n')
    file.write('Train Results\n')
    for result in train_results:
        file.write(str(result) + '\n')
    file.close()

In [14]:
b = [[1,2,3],[1,2,3],[1,2,3]]
a = [[1,2,3],[1,2,3],[1,2,3]]
save_as_txt(a,b,'./Results/test.txt')